In [1]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
zomato_pipline=pd.read_csv('C:\\Users\\tasne\\Downloads\\intern-epsilon\\zomato.csv\\zomato.csv')


In [2]:
zomato_pipline.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'listed_type','listed_in(city)':'listed_city'},inplace=True)

In [3]:
zomato_pipline['rate'] = zomato_pipline['rate'].str.replace('/5','')

In [4]:
zomato_pipline['rate'] = zomato_pipline['rate'].str.replace('-','0')

In [5]:
zomato_pipline['rate'] = zomato_pipline['rate'].str.replace('NEW','0')

In [6]:
zomato_pipline['rate'] = zomato_pipline['rate'].astype('float')

In [7]:
zomato_pipline.drop(columns=['url','address','phone','reviews_list','menu_item','dish_liked'],inplace=True)

In [8]:
def target(x):
  if (x>=3.7):
    return'succsses'
  else :
    return 'not succsses'  

In [9]:
zomato_pipline['Target'] = zomato_pipline['rate'].apply(target)

In [10]:
#zomato_pipline.drop('rate',inplace=True,axis=1)

In [11]:
zomato_pipline['location'].isna().sum()

21

In [12]:
zomato_pipline['location'].dropna(inplace=True)

In [13]:
from sklearn.tree import DecisionTreeClassifier

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X=zomato_pipline.drop('Target',axis=1)
y=zomato_pipline['Target']

In [16]:
x_train,x_test,y_train,y_test =train_test_split(X,y,random_state=42,test_size=0.2,stratify=y)

In [17]:
numeric_columns = x_train.select_dtypes(exclude='object').columns

In [18]:
categorical_columns = x_train.select_dtypes(include='object').columns

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
numerical_features = Pipeline(steps=[('handeling missing values',SimpleImputer(strategy='median')),('scaling',StandardScaler(with_mean=False))])

In [20]:
numerical_features

Pipeline(steps=[('handeling missing values', SimpleImputer(strategy='median')),
                ('scaling', StandardScaler(with_mean=False))])

In [21]:
from sklearn.preprocessing import OneHotEncoder
categorical_features = Pipeline(steps=[('handeling missing values',SimpleImputer(strategy='most_frequent')),
                                      ('handle cat features',OneHotEncoder(handle_unknown='ignore',drop='first')),('scaling',StandardScaler(with_mean=False))])

In [22]:
categorical_features

Pipeline(steps=[('handeling missing values',
                 SimpleImputer(strategy='most_frequent')),
                ('handle cat features',
                 OneHotEncoder(drop='first', handle_unknown='ignore')),
                ('scaling', StandardScaler(with_mean=False))])

In [23]:
from sklearn.compose import ColumnTransformer
preprocessing = ColumnTransformer(transformers=[('numeric',numerical_features,numeric_columns),('categorical',categorical_features,categorical_columns)],remainder='passthrough')

In [24]:
final_pipline = Pipeline(steps=[('preprocessing',preprocessing),('modeling',DecisionTreeClassifier(max_depth= 4, criterion='entropy'))])

In [25]:
final_pipline.fit(x_train,y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numeric',
                                                  Pipeline(steps=[('handeling '
                                                                   'missing '
                                                                   'values',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaling',
                                                                   StandardScaler(with_mean=False))]),
                                                  Index(['rate', 'votes'], dtype='object')),
                                                 ('categorical',
                                                  Pipeline(steps=[('handeling '
                                                                   'missing '
           

In [26]:
final_pipline.predict(x_test)

C:\Users\tasne\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [0, 4, 5] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array(['not succsses', 'succsses', 'not succsses', ..., 'not succsses',
       'succsses', 'succsses'], dtype=object)

In [27]:
import joblib
joblib.dump(final_pipline,'zomato_pipeline_pipeline_test1.pkl')

['zomato_pipeline_pipeline_test1.pkl']

In [28]:
model = joblib.load('zomato_pipeline_pipeline_test1.pkl')

In [29]:
prediction = model.predict(pd.DataFrame({'name':['Jalsa'],
                                         'online_order':['Yes'], 'book_table':['Yes'],'rate':[4.1],
                                             'votes':[775],'location':['Banashankari'],
                                             'rest_type':['Casual Dining'],
                                             'cuisines':['North Indian, Mughlai, Chinese'],
                                             'cost':[800],
                                            'listed_type':['Buffet'],
                                        'listed_city':['Banashankari']}))
prediction

C:\Users\tasne\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:170: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


array(['succsses'], dtype=object)

In [30]:
zomato_pipline.columns

Index(['name', 'online_order', 'book_table', 'rate', 'votes', 'location',
       'rest_type', 'cuisines', 'cost', 'listed_type', 'listed_city',
       'Target'],
      dtype='object')

In [31]:
%%writefile zomato_final_model_edit.py
import pandas as pd
import numpy as np
import joblib
import streamlit as st
import sklearn
import base64

model = joblib.load('zomato_pipeline_pipeline_test1.pkl')
def predict_Target(name, online_order, book_table,rate, votes,
       location, rest_type,cuisines,cost,
       listed_type, listed_city):
    
    prediction = model.predict(pd.DataFrame({'online_order':online_order,
                                             'book_table':book_table,'rate':rate,
                                             'votes':votes,'location':location,
                                             'rest_type':rest_type,
                                             'cuisines':cuisines,
                                             'cost':cost,
                                             'listed_type':listed_type,
                                             'listed_city':listed_city}))
    label=['succsses','not succsses']
        
    return  label[prediction[0]]
    
def main():
    def add_bg_from_local(image_file):
        with open(image_file, "rb") as image_file:
            encoded_string = base64.b64encode(image_file.read())
        st.markdown(
        f"""
            <style>
            .stApp {{
                background-image: url(data:image/{"jpg"};base64,{encoded_string.decode()});
                background-size: cover
            }}
            </style>
            """,
            unsafe_allow_html=True
            )
    add_bg_from_local("C:\\Users\\tasne\\Downloads\\On.jpg")    
      
    st.title('succsess Prediction')
   
    html_temp = """
        
        <div style='background-color:white'>
        <h2 style="color:red;text-align:center;">resturants succsess</h2>
        </div>
                  """
    
    st.markdown(html_temp,unsafe_allow_html=True)
   
    name=st.text_input('name','name')
    online_order = st.selectbox('online_order',['Yes','No'])
    book_table = st.selectbox('book_table',['Yes','No'])
    rate=st.slider('rate',0,5)
    votes = st.text_input('votes','votes')
    location = st.text_input('location','location')
    rest_type=st.text_input('rest_type','rest_type')
    cuisines = st.text_input('cuisines','cuisines')
    cost=st.slider('cost',100,3000)
    listed_type = st.text_input('listed_type','listed_type')
    listed_city=st.text_input('listed_city','listed_city')               
    result = ""
    if st.button('predict'):
       result = predict_Target(name, online_order, book_table,rate, votes,
                location, rest_type,cuisines, cost,
                listed_type, listed_city)
    st.success('The Target is {} '.format(result)) 

    

if __name__ =='__main__':
    main()
        

Overwriting zomato_final_model_edit.py


In [ ]:
!streamlit run zomato_final_model_edit.py